In [1]:
from pathlib import Path
from scripts.data.data_acquisition import DataAcquisition

In [3]:
PATH_DATA = Path("data")

data = DataAcquisition(path_data = PATH_DATA)
# En caso de requerir continuar con la ejecución del proceso
# desde cierta categoría en específico, se puede hacer uso
# del siguiente código para eliminar las categorías ya procesadas
# last_cat_index = prueba.cats[prueba.cats["id"]=="MCO40433"].index[0]
# prueba.cats = prueba.cats[prueba.cats.index >= last_cat_index]

# 41 minutos
data.get_all_items_cats(0, 3950)